In [4]:
!pip install -U numpy
!pip install tqdm
!pip install python-igraph -U
!pip install gensim
!pip install cxrandomwalk
!pip install xnetwork
!pip install pandas
!pip install matplotlib -U
!pip install --user numba llvmlite -U
!pip install umap-learn -U

# Need to restart kernel after that

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Using cached numpy-1.20.3-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.3 MB)
ERROR: numba 0.48.0 has requirement llvmlite<0.32.0,>=0.31.0dev0, but you'll have llvmlite 0.36.0 which is incompatible.
  Attempting uninstall: numpy
    Found existing installation: numpy 1.17.5
    Uninstalling numpy-1.17.5:
      Successfully uninstalled numpy-1.17.5
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Using cached python_igraph-0.9.6-cp37-cp37m-manylinux2010_x86_64.whl (3.2 M

In [1]:
from tqdm.auto import tqdm
import pandas as pd
import xnetwork as xnet
import numpy as np
import igraph as ig
import sys
import os
import cxrandomwalk as rw
import umap
import gensim
import importlib
import matplotlib
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from collections import OrderedDict
%matplotlib inline

queryID = "ab3f89ab-1d62-449e-a1c1-a8457e88cd00"


/opt/conda/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [4]:
!jupyter labextension install @jupyter-widgets/jupyterlab-manager jupyter-matplotlib@0.7.0

An error occured.
ValueError: "@jupyter-widgets/jupyterlab-manager" is not a valid extension:
schemaDir is empty: "./schema"
See the log file for details:  /tmp/jupyterlab-debug-v8bavboc.log


In [5]:
plt.figure()
plt.plot()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
os.makedirs("networks",exist_ok=True)
networkPath = "networks/"+queryID+".xnet"

os.makedirs("models", exist_ok=True)
modelPath = "models/%s.model"%queryID

os.makedirs("sentences", exist_ok=True)
sentencesPath = "sentences/%s.txt"%queryID

In [2]:
MAGColumnTypes = {
    "Paper_paperId":int,
    "journal_id": object,
    "Paper_year": object,
    "Paper_originalTitle": object,
    "Author_displayName": object,
    "JournalFixed_displayName": object,
    "isQueryPaper": object,
}

In [6]:

def mag_query_input_to_xnet(nodes_file, edges_file, output_file):
    global edgesData,nodesData,vertexAttributes,index2ID
    edgesData = pd.read_csv(edges_file)
    nodesData = pd.read_csv(nodes_file, dtype=MAGColumnTypes)

    # Replacing NaN for empty string
    for key in MAGColumnTypes:
        if(key in nodesData):
            nodesData[key].fillna("",inplace=True)

    # Generating continous indices for papers
    index2ID  = nodesData["Paper_paperId"].tolist()
    ID2Index = {id:index for index, id in enumerate(index2ID)}


    # Hack to account for 2 degree capitalized "FROM"
    fromKey = "From (Citing)"

    toKey = "To (Cited)"
    
    # Converting edges from IDs to new indices
    # Invert edges so it means a citation between from to to
    edgesZip = zip(edgesData[fromKey].tolist(),edgesData[toKey].tolist())
    edgesList = [(ID2Index[toID],ID2Index[fromID]) for fromID,toID in edgesZip if fromID in ID2Index and toID in ID2Index]

    vertexAttributes = {key:nodesData[key].tolist() for key in nodesData}

    for key in nodesData:
        nodesData[key].tolist()

    graph = ig.Graph(
        n=len(index2ID),
        edges=edgesList,
        directed=True,
        vertex_attrs=vertexAttributes
    )

    # verticesToDelete = np.where(np.logical_or(np.array(graph.indegree())==0,np.array(graph.degree())==0))[0]
    # graph.delete_vertices(verticesToDelete)
    graph.vs["KCore"] = graph.shell_index(mode="IN")

    if("Paper_year" in graph.vertex_attributes()):
        graph.vs["year"] = [int(year) for year in graph.vs["Paper_year"]]
    else:
        graph.vs["year"] = [int(s[0:4]) for s in graph.vs["date"]]

    giantComponent = graph.clusters(mode="WEAK").giant()
    giantCopy = giantComponent.copy()
    giantCopy.to_undirected()
    giantComponent.vs["Community"] = [str(c) for c in giantCopy.community_multilevel().membership]
    xnet.igraph2xnet(giantComponent, output_file)

In [7]:
mag_query_input_to_xnet(
        "query-results/%s.csv"%queryID,
        "query-results/%s_edges.csv"%queryID,
        networkPath
    )

In [8]:
from gensim.models.callbacks import CallbackAny2Vec

class MonitorCallback(CallbackAny2Vec):
    def __init__(self,pbar):
        self.pbar = pbar

    def on_epoch_end(self, model):
        self.pbar.update(1)
#         self.pbar.set_description("loss: %g"%model.get_latest_training_loss())
        self.pbar.refresh() # to show immediately the update
    
    


In [9]:
g = xnet.xnet2igraph(networkPath)

In [7]:
agent = rw.Agent(g.vcount(),np.array(g.get_edgelist()),False);

def make_pbar():
    pbar = None
    def inner(current,total):
        nonlocal pbar
        if(pbar is None):
            pbar= tqdm(total=total);
        pbar.update(current - pbar.n)
    return inner

agent.generateWalks(q=1.0,p=1.0,filename=sentencesPath,walksPerNode=5,verbose=False,updateInterval=100,callback=make_pbar()) #filename="entireData.txt",


In [10]:


monitor = MonitorCallback(tqdm())
gensimModel = gensim.models.Word2Vec(
    gensim.models.word2vec.LineSentence(sentencesPath),
    vector_size=64,
    workers=2,
    min_count=1,
    sg=1,
    callbacks=[monitor]
)#,negative=10)

gensimModel.save(modelPath)


KeyboardInterrupt: 

In [11]:
gensimModel = gensim.models.Word2Vec.load(modelPath)
gensimVector = gensimModel.wv;
# reducer = umap.UMAP(n_neighbors=16, min_dist=0.15, n_components=2, metric='cosine')
reducer = umap.UMAP(n_neighbors=10, min_dist=0.25, n_components=2, metric='cosine')
embedding = reducer.fit_transform(gensimVector.vectors)

keyindex = {int(entry):i for i,entry in enumerate(gensimVector.index_to_key)}
indexkey = [keyindex[index] for index in sorted(keyindex.keys())]
correctEmbedding = embedding[indexkey,:]

g.vs['Position'] = correctEmbedding
xnet.igraph2xnet(g,"networks/"+queryID+"_positions.xnet")

In [13]:


def adjust_lightness(color, amount=0.5):
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], amount, c[2])
communities = g.vs["Community"]
sortedTitles = sorted(list(set(communities)))
c2i = {c:i if i<10 else 10 for i,c in enumerate(sortedTitles)}
c2cc = {c:cm.tab10(i)[0:3]+(0.25,) if i<10 else (0.75,0.75,0.75,0.05) for i,c in enumerate(sortedTitles)} 
c2ccLight = {c:adjust_lightness(cm.tab10(i)[0:3]+(0.25,),0.98) if i<10 else adjust_lightness((0.75,0.75,0.75,0.05),0.98) for i,c in enumerate(sortedTitles)} 

In [ ]:
fig = plt.figure(figsize=(15, 15),dpi=300)

ax = fig.add_axes([0, 0, 1, 1])
colors = [c2cc[entry] for entry in communities];
colorsLight = [c2ccLight[entry] for entry in communities];
# plt.scatter(
#     correctEmbedding[invalidIndices, 0],
#     correctEmbedding[invalidIndices, 1],
# #     s=10*np.array(journalRankings)/np.max(journalRankings),
#     # alpha=[0.2 if c2i[entry]<10 else 0.05 for entry in communities],
#     s=[1 if c2i[communities[entryIndex]]<10 else 0.5 for entryIndex in invalidIndices],
#     # # cmap = "tab10",
#     c=[colorsLight[entryIndex] for entryIndex in invalidIndices],
#     rasterized=True,
# )
plt.scatter(
  correctEmbedding[:, 0],
  correctEmbedding[:, 1],
#     s=10*np.array(journalRankings)/np.max(journalRankings),
  # alpha=[0.2 if c2i[entry]<10 else 0.05 for entry in communities],
#   s=[1 if c2i[communities[entryIndex]]<10 else 0.5 for entryIndex in validIndices],
  # # cmap = "tab10",
  c=colors ,
  rasterized=True
)


plt.setp(ax, xticks=[], yticks=[]);
fig.patch.set_visible(False)
ax.axis('off')
plt.savefig("map.pdf",dpi=600);
plt.savefig("map.png",dpi=150);
plt.close();

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15),dpi=300)
legend_elements = [Line2D([0], [0],
                    linewidth=0,
                    marker='o',
                    color=c2cc[community][0:3],
                    label=", ".join(community.replace(", %,",",",).replace(", =,",",").split(", ")[0:7]),
                    # markerfacecolor='g',
                    markersize=10) for community in sortedTitles[0:10]]
legend_elements+=[Line2D([0], [0],
                    linewidth=0,
                    marker='o',
                    color=(0.75,0.75,0.75),
                    label="Others",
                    # markerfacecolor='g',
                    markersize=10)]
ax.legend(handles=legend_elements, loc='center')
plt.setp(ax, xticks=[], yticks=[]);
plt.savefig("legends.pdf");
plt.close();